In [ ]:
pip install requests tqdm pillow pyyaml


# Import the file annotations.json

In [ ]:
import os
import json
import requests
import random
import yaml
from tqdm import tqdm
from PIL import Image

# === Load TACO annotations ===
ANNOTATIONS_FILE = "/content/annotations.json"
OUTPUT_DIR = "./TACO_YOLO"  # path where you want to save
TRAIN_RATIO = 1450  # Number of images for training
VAL_RATIO = 50      # Number of images for validation

# === Create output directories ===
os.makedirs(f"{OUTPUT_DIR}/train/images", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/train/labels", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/val/images", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/val/labels", exist_ok=True)

# === Load JSON data ===
with open(ANNOTATIONS_FILE, "r") as f:
    taco_data = json.load(f)

images = taco_data["images"]
annotations = taco_data["annotations"]
categories = taco_data["categories"]

# === Map category IDs to names ===
category_id_to_name = {cat["id"]: cat["name"] for cat in categories}
category_id_to_index = {cat["id"]: i for i, cat in enumerate(categories)}

# === Prepare Image List ===
random.shuffle(images)  # Shuffle images before splitting
train_images = images[:TRAIN_RATIO]
val_images = images[TRAIN_RATIO:TRAIN_RATIO + VAL_RATIO]

# === Function to download images ===
def download_image(image_info, save_path):
    url = image_info.get("flickr_640_url") or image_info.get("flickr_url")
    if not url:
        return False

    # Ensure parent directory exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    try:
        response = requests.get(url, stream=True, timeout=10)
        if response.status_code == 200:
            with open(save_path, "wb") as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            return True
    except requests.RequestException:
        return False
    return False

# === Process and save images & annotations in YOLO format ===
def process_dataset(image_list, split):
    for image in tqdm(image_list, desc=f"Processing {split} images"):
        image_id = image["id"]
        file_name = image["file_name"]
        width, height = image["width"], image["height"]

        # Ensure subdirectories exist
        image_save_path = f"{OUTPUT_DIR}/{split}/images/{file_name}"
        os.makedirs(os.path.dirname(image_save_path), exist_ok=True)

        # Download and save image
        if not download_image(image, image_save_path):
            print(f"Skipping {file_name} (Failed to download)")
            continue

        # Filter annotations for this image
        image_annotations = [ann for ann in annotations if ann["image_id"] == image_id]
        if not image_annotations:
            continue  # Skip images with no annotations

        # Convert annotations to YOLO format
        yolo_annotations = []
        for ann in image_annotations:
            bbox = ann["bbox"]
            category_id = ann["category_id"]

            # Convert bbox [x, y, width, height] -> [x_center, y_center, width, height] (normalized)
            x, y, w, h = bbox
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            w /= width
            h /= height

            yolo_annotations.append(f"{category_id_to_index[category_id]} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

        # Ensure label directory exists
        label_save_path = f"{OUTPUT_DIR}/{split}/labels/{file_name.replace('.jpg', '.txt').replace('.png', '.txt')}"
        os.makedirs(os.path.dirname(label_save_path), exist_ok=True)

        # Save annotation file
        with open(label_save_path, "w") as f:
            f.write("\n".join(yolo_annotations))

# === Process train and validation datasets ===
process_dataset(train_images, "train")
process_dataset(val_images, "val")

# === Create dataset.yaml for YOLOv11 training ===
dataset_yaml = {
    "train": f"/content/TACO_YOLO/train/images",
    "val": f"/content/TACO_YOLO/train/images",
    "nc": len(category_id_to_index),
    "names": [category_id_to_name[i] for i in sorted(category_id_to_index.keys())]
}

# Save YAML file
yaml_path = f"{OUTPUT_DIR}/taco_yolo.yaml"
with open(yaml_path, "w") as f:
    yaml.dump(dataset_yaml, f, default_flow_style=False)

print(f"✅ Dataset preparation complete! YOLO dataset saved in {OUTPUT_DIR}")
print(f"✅ Dataset configuration saved at: {yaml_path}")


Processing val images: 100%|██████████| 50/50 [00:26<00:00,  1.91it/s]

✅ Dataset preparation complete! YOLO dataset saved in ./TACO_YOLO
✅ Dataset configuration saved at: ./TACO_YOLO/taco_yolo.yaml


In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.9/921.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# Step 2: Define the folder path
folder_path = "/content/drive/My Drive/YOLOv11-litter-detec-folder"

# Step 3: Check contents of the folder
import os
if os.path.exists(folder_path):
    print(f"Contents of {folder_path}:")
    print(os.listdir(folder_path))
else:
    print(f"Folder {folder_path} does not exist. Check the path.")

Mounted at /content/drive/
Contents of /content/drive/My Drive/YOLOv11-litter-detec-folder:
['yolo11m.pt', 'dataset_custom.yaml', 'yolo11m-custom-litter-best.pt', 'yolo11x.pt', 'yolo11x-custom-litter-best.pt', 'yolo11n.pt', 'yolo11n-custom-litter-best.pt', 'vecteezy_a-city-s-garbage_1803529.gif', 'val', 'train']


In [ ]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/YOLOv11-litter-detec-folder/yolo11m.pt")  # Or use `yolo11x.pt`
model.train(
    data="TACO_YOLO/taco_yolo.yaml",
    imgsz=640,
    batch=16,
    epochs=100,
    workers=8,
    device=0,  # Use 0 for GPU or "cpu"
    val=True
)


Ultralytics 8.3.80 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/YOLOv11-litter-detec-folder/yolo11m.pt, data=TACO_YOLO/taco_yolo.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

100%|██████████| 755k/755k [00:00<00:00, 14.6MB/s]


Overriding model.yaml nc=80 with nc=60

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 67.0MB/s]


AMP: checks passed ✅


...

                   all         50         76      0.549      0.194      0.273       0.21



100 epochs completed in 0.460 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 40.6MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 40.6MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.80 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11m summary (fused): 125 layers, 20,076,292 parameters, 0 gradients, 67.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  7.91it/s]


                   all         50         76      0.409      0.224      0.293      0.238
  Other plastic bottle          1          1      0.902          1      0.995      0.895
  Clear plastic bottle          2          2      0.122        0.5      0.548      0.474
          Glass bottle          1          2          0          0      0.497      0.156
    Plastic bottle cap          3          3      0.217      0.333      0.344      0.108
      Metal bottle cap          1          1      0.528          1      0.995      0.995
             Drink can          4          4      0.341       0.75      0.468      0.383
          Other carton          1          1          0          0     0.0524     0.0367
          Drink carton          1          1          1          0          0          0
     Corrugated carton          2          3      0.173      0.333       0.43      0.151
           Meal carton          1          1          1          0          0          0
           Plastic li

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 4,  5,  6,  7,  8, 12, 14, 16, 17, 18, 27, 29, 31, 33, 36, 39, 42, 48, 50, 51, 52, 57, 58, 59])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c6560127dd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.

In [ ]:
import shutil

# Specify the folder or directory you want to zip
folder_to_zip = "/content/runs"

# Specify the name of the output ZIP file
output_zip_file = "/content/runs.zip"

# Create the ZIP file
shutil.make_archive(base_name=output_zip_file.replace(".zip", ""), format="zip", root_dir=folder_to_zip)

print(f"ZIP file created: {output_zip_file}")

ZIP file created: /content/runs.zip


In [ ]:
# predict

from ultralytics import YOLO

model = YOLO("/content/best.pt")

model.predict(
    source="/content/000005.jpg",
    # show = True,
    save = True,
    # conf = 0.6,
    line_width = 4,
    save_crop = False, # to crop and save only the litter img
    show_labels = True,
    show_conf = True,
    # classes = [0,1,2,3,4] # predict only the classes 0 upto 4
)


image 1/1 /content/000005.jpg: 640x480 1 Single-use carrier bag, 1 Unlabeled litter, 14.1ms
Speed: 3.7ms preprocess, 14.1ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict5


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Aluminium foil', 1: 'Battery', 2: 'Aluminium blister pack', 3: 'Carded blister pack', 4: 'Other plastic bottle', 5: 'Clear plastic bottle', 6: 'Glass bottle', 7: 'Plastic bottle cap', 8: 'Metal bottle cap', 9: 'Broken glass', 10: 'Food Can', 11: 'Aerosol', 12: 'Drink can', 13: 'Toilet tube', 14: 'Other carton', 15: 'Egg carton', 16: 'Drink carton', 17: 'Corrugated carton', 18: 'Meal carton', 19: 'Pizza box', 20: 'Paper cup', 21: 'Disposable plastic cup', 22: 'Foam cup', 23: 'Glass cup', 24: 'Other plastic cup', 25: 'Food waste', 26: 'Glass jar', 27: 'Plastic lid', 28: 'Metal lid', 29: 'Other plastic', 30: 'Magazine paper', 31: 'Tissues', 32: 'Wrapping paper', 33: 'Normal paper', 34: 'Paper bag', 35: 'Plastified paper bag', 36: 'Plastic film', 37: 'Six pack rings', 38: 'Garbage bag', 39: 'Other plastic wrapper', 40: 'Single-use carrier 

In [ ]:
import os

print("Checking dataset directories...")
print("Train images:", len(os.listdir("TACO_YOLO/train/images")) if os.path.exists("TACO_YOLO/train/images") else "Not found")
print("Train labels:", len(os.listdir("TACO_YOLO/train/labels")) if os.path.exists("TACO_YOLO/train/labels") else "Not found")
print("Val images:", len(os.listdir("TACO_YOLO/val/images")) if os.path.exists("TACO_YOLO/val/images") else "Not found")
print("Val labels:", len(os.listdir("TACO_YOLO/val/labels")) if os.path.exists("TACO_YOLO/val/labels") else "Not found")


Checking dataset directories...
Train images: 15
Train labels: 15
Val images: 14
Val labels: 14
